In [4]:
from pathlib import Path

import lightgbm as lgb
import mlforecast
import numpy as np
import pandas as pd
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean, SeasonalRollingMean

In [71]:
import mlforecast

AttributeError: module 'coreforecast.lag_transforms' has no attribute 'BaseLagTransform'

In [69]:
pip install coreforecast==0.0.2 --user

  Using cached coreforecast-0.0.2-py3-none-win_amd64.whl (41 kB)



In [2]:
mlforecast.__version__

'0.11.2'

In [80]:
pip install "mlforecast-0.11.2.tar.gz"

Processing c:\users\lywt9\downloads\mlforecast-0.11.2.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for mlforecast: filename=mlforecast-0.11.2-py3-none-any.whl size=53816 sha256=084fc14bb07e28aaeefd5acd7ae9f3b4dd180ef034ae86f94e5e6b47125b2127
  Stored in directory: c:\users\lywt9\appdata\local\pip\cache\wheels\fb\2e\89\d5b58e941a901af50f5a264e18dcaf6b818a17f4428365e093
Successfully built mlforecast
  Attempting uninstall: mlforecast
    Found existing installation: mlforecast 0.11.2
    Uninstalling mlforecast-0.11.2:
      Successfully uninstalled mlforecast-0.11.2
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.3 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.


In [ ]:
%pip install -qqq "mlforecast[lag_transforms]"

In [ ]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')

In [ ]:
sell_prices=pd.read_csv('sell_prices.csv')

In [ ]:
calendar=pd.read_csv('calendar.csv')

# LightBGM

In [2]:
import lightgbm as lgb

In [7]:
model_params = {
    'verbose': -1,
    'force_col_wise': True,
    'num_leaves': 256,
    'n_estimators': 50,
}

In [8]:
sales_train=pd.read_csv('sales_train_evaluation.csv')

In [9]:
cal_dtypes = {
    'd': 'category',
    'wm_yr_wk': np.uint16,
    'event_name_1': 'category',
    'event_type_1': 'category',
    'event_name_2': 'category',
    'event_type_2': 'category',
    'snap_CA': np.uint8,
    'snap_TX': np.uint8,
    'snap_WI': np.uint8,
}
cal = pd.read_csv('calendar.csv', 
                  dtype=cal_dtypes, 
                  usecols=list(cal_dtypes.keys()) + ['date'], 
                  parse_dates=['date'])


In [10]:
event_cols = [k for k in cal_dtypes if k.startswith('event')]

In [ ]:
event_cols

In [11]:

for col in event_cols:
    cal[col] = cal[col].cat.add_categories('nan').fillna('nan')

In [ ]:
cal

In [12]:
prices_dtypes = {
    'store_id': 'category',
    'item_id': 'category',
    'wm_yr_wk': np.uint16,
    'sell_price': np.float32
}
prices = pd.read_csv('sell_prices.csv', dtype=prices_dtypes)

In [13]:
sales_dtypes = {
    'id': 'category',
    'item_id': prices.item_id.dtype,
    'dept_id': 'category',
    'cat_id': 'category',
    'store_id': 'category',
    'state_id': 'category',
    **{f'd_{i}': np.float32 for i in range(1942)}
}
sales = pd.read_csv(
     'sales_train_evaluation.csv',
    dtype=sales_dtypes,
)

In [ ]:
sales

In [14]:
long = sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

In [ ]:
long

In [15]:
%%time
print(long.shape[0])
long['date_idx'] = long['d'].str.replace('d_', '').astype('int32')








59181090
Wall time: 1min 14s


In [ ]:
long

In [16]:
dates = sorted(long['date_idx'].unique())

In [ ]:
dates

In [17]:
long = long.sort_values(['id', 'date_idx'])

In [47]:
long

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date_idx
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,1
30490,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0.0,2
60980,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0.0,3
91470,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0.0,4
121960,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0.0,5
...,...,...,...,...,...,...,...,...,...
59059129,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1937,0.0,1937
59089619,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1938,2.0,1938
59120109,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1939,2.0,1939
59150599,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1940,5.0,1940


In [18]:
without_leading_zeros = long['y'].gt(0).groupby(long['id']).transform('cummax')

In [49]:
without_leading_zeros

0           False
30490       False
60980       False
91470       False
121960      False
            ...  
59059129     True
59089619     True
59120109     True
59150599     True
59181089     True
Name: y, Length: 59181090, dtype: bool

In [19]:
above_min_date = long['date_idx'] >= dates[-400]

In [51]:
above_min_date

0           False
30490       False
60980       False
91470       False
121960      False
            ...  
59059129     True
59089619     True
59120109     True
59150599     True
59181089     True
Name: date_idx, Length: 59181090, dtype: bool

In [20]:
keep_mask = without_leading_zeros & above_min_date

In [ ]:
keep_mask

In [ ]:
long

In [21]:
long = long[keep_mask]

In [ ]:
long

In [ ]:
print(long.shape[0])

In [22]:
long['d'] = long['d'].astype(cal.d.dtype)


In [23]:
long = long.merge(cal, on=['d'])


In [ ]:
long

In [24]:
long = long.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'])


In [ ]:
long

In [25]:
last_wmyrwk = long['wm_yr_wk'].max()


In [ ]:
last_wmyrwk

In [26]:
last_date_train = long['date'].max()


In [ ]:
last_date_train

In [27]:
long = long.drop(columns=['d', 'date_idx', 'wm_yr_wk'])


In [ ]:
long

In [28]:
long = long.sample(frac=1.0, random_state=0).reset_index(drop=True)



In [ ]:
long

In [29]:
# build future X
future_cal = cal[cal['date'] > last_date_train]


In [ ]:
future_cal

In [30]:
future_prices = prices[prices['wm_yr_wk'] >= last_wmyrwk].copy()


In [ ]:
future_prices 

In [31]:
future_prices['id'] = future_prices['item_id'].astype(str) + '_' + future_prices['store_id'].astype(str) + '_evaluation'


In [ ]:
future_prices 

In [32]:
X_df = future_prices.merge(future_cal, on='wm_yr_wk').drop(columns=['store_id', 'item_id', 'wm_yr_wk', 'd'])

In [33]:
X_df

,sell_price,id,date,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,8.38,HOBBIES_1_001_CA_1_evaluation,2016-05-23,nan,nan,nan,nan,0,0,0
1,8.38,HOBBIES_1_001_CA_1_evaluation,2016-05-24,nan,nan,nan,nan,0,0,0
2,8.38,HOBBIES_1_001_CA_1_evaluation,2016-05-25,nan,nan,nan,nan,0,0,0
3,8.38,HOBBIES_1_001_CA_1_evaluation,2016-05-26,nan,nan,nan,nan,0,0,0
4,8.38,HOBBIES_1_001_CA_1_evaluation,2016-05-27,nan,nan,nan,nan,0,0,0
...,...,...,...,...,...,...,...,...,...,...
853715,3.98,FOODS_3_825_WI_3_evaluation,2016-06-19,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0
853716,1.28,FOODS_3_826_WI_3_evaluation,2016-06-18,nan,nan,nan,nan,0,0,0
853717,1.28,FOODS_3_826_WI_3_evaluation,2016-06-19,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0
853718,1.00,FOODS_3_827_WI_3_evaluation,2016-06-18,nan,nan,nan,nan,0,0,0


In [34]:
model_params = {
    'verbose': -1,
    'force_col_wise': True,
    'num_leaves': 256,
    'n_estimators': 50,
}

fcst = MLForecast(
    models=[lgb.LGBMRegressor(**model_params)],
    freq='D',
    lags=[7 * (i+1) for i in range(8)],
    lag_transforms = {
        1:  [ExpandingMean()],
        7:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        14: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        28: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
    },
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week'],    
    num_threads=4,
)

In [35]:
%%time
fcst.fit(
    long,
    id_col='id',
    time_col='date',
    target_col='y',
    static_features=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
)

Wall time: 4min 51s


MLForecast(models=[LGBMRegressor], freq=D, lag_features=['lag7', 'lag14', 'lag21', 'lag28', 'lag35', 'lag42', 'lag49', 'lag56', 'expanding_mean_lag1', 'rolling_mean_lag7_window_size7', 'rolling_mean_lag7_window_size14', 'rolling_mean_lag7_window_size28', 'seasonal_rolling_mean_lag7_season_length7_window_size4', 'rolling_mean_lag14_window_size7', 'rolling_mean_lag14_window_size14', 'rolling_mean_lag14_window_size28', 'seasonal_rolling_mean_lag14_season_length7_window_size4', 'rolling_mean_lag28_window_size7', 'rolling_mean_lag28_window_size14', 'rolling_mean_lag28_window_size28', 'seasonal_rolling_mean_lag28_season_length7_window_size4'], date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week'], num_threads=4)

In [36]:
mlforecast.__version__


'0.11.2'

In [37]:
%time preds = fcst.predict(28, X_df=X_df)

ValueError: Found missing inputs in X_df. It should have one row per id and time for the complete forecasting horizon.
You can get the expected structure by running `MLForecast.make_future_dataframe(h)` or get the missing combinatins in your current `X_df` by running `MLForecast.get_missing_future(h, X_df)`.

In [ ]:
wide = preds.pivot_table(index='id', columns='date')
wide.columns = [f'F{i+1}' for i in range(28)]
wide.columns.name = None
wide.index.name = 'id'
wide

In [ ]:
%pip install -qqq "mlforecast[lag_transforms]"

In [ ]:
from pathlib import Path

import lightgbm as lgb
import mlforecast
import numpy as np
import pandas as pd
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean, SeasonalRollingMean

In [ ]:
mlforecast.__version__

In [ ]:
cal_dtypes = {
    'd': 'category',
    'wm_yr_wk': np.uint16,
    'event_name_1': 'category',
    'event_type_1': 'category',
    'event_name_2': 'category',
    'event_type_2': 'category',
    'snap_CA': np.uint8,
    'snap_TX': np.uint8,
    'snap_WI': np.uint8,
}
cal = pd.read_csv('calendar.csv', 
                  dtype=cal_dtypes, 
                  usecols=list(cal_dtypes.keys()) + ['date'], 
                  parse_dates=['date'])
event_cols = [k for k in cal_dtypes if k.startswith('event')]
for col in event_cols:
    cal[col] = cal[col].cat.add_categories('nan').fillna('nan')

In [ ]:
prices_dtypes = {
    'store_id': 'category',
    'item_id': 'category',
    'wm_yr_wk': np.uint16,
    'sell_price': np.float32
}
prices = pd.read_csv( 'sell_prices.csv', dtype=prices_dtypes)

In [ ]:
prices_dtypes = {
    'store_id': 'category',
    'item_id': 'category',
    'wm_yr_wk': np.uint16,
    'sell_price': np.float32
}
prices = pd.read_csv('sell_prices.csv', dtype=prices_dtypes)

In [ ]:
long = sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

In [ ]:
%%time
print(long.shape[0])
long['date_idx'] = long['d'].str.replace('d_', '').astype('int32')
dates = sorted(long['date_idx'].unique())
long = long.sort_values(['id', 'date_idx'])
without_leading_zeros = long['y'].gt(0).groupby(long['id']).transform('cummax')
above_min_date = long['date_idx'] >= dates[-400]
keep_mask = without_leading_zeros & above_min_date
long = long[keep_mask]
print(long.shape[0])

In [ ]:
long['d'] = long['d'].astype(cal.d.dtype)
long = long.merge(cal, on=['d'])
long = long.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'])
last_wmyrwk = long['wm_yr_wk'].max()
last_date_train = long['date'].max()
long = long.drop(columns=['d', 'date_idx', 'wm_yr_wk'])
long = long.sample(frac=1.0, random_state=0).reset_index(drop=True)

# build future X
future_cal = cal[cal['date'] > last_date_train]
future_prices = prices[prices['wm_yr_wk'] >= last_wmyrwk].copy()
future_prices['id'] = future_prices['item_id'].astype(str) + '_' + future_prices['store_id'].astype(str) + '_evaluation'
X_df = future_prices.merge(future_cal, on='wm_yr_wk').drop(columns=['store_id', 'item_id', 'wm_yr_wk', 'd'])

In [ ]:
X_df = future_prices.merge(future_cal, on='wm_yr_wk').drop(columns=['store_id', 'item_id', 'wm_yr_wk', 'd'])

In [ ]:
model_params = {
    'verbose': -1,
    'force_col_wise': True,
    'num_leaves': 256,
    'n_estimators': 50,
}

fcst = MLForecast(
    models=[lgb.LGBMRegressor(**model_params)],
    freq='D',
    lags=[7 * (i+1) for i in range(8)],
    lag_transforms = {
        1:  [ExpandingMean()],
        7:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        14: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        28: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
    },
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week'],    
    num_threads=4,
)

In [ ]:
%%time
fcst.fit(
    long,
    id_col='id',
    time_col='date',
    target_col='y',
    static_features=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
)

In [ ]:
%time preds = fcst.predict(28, X_df=X_df)

In [ ]:
fcst.make_future_dataframe(28)

In [ ]:
x=fcst.get_missing_future(28, X_df)

In [ ]:
fcst.get_missing_future(28, X_df)

In [ ]:
x[x['id']=='HOBBIES_2_132_CA_1_evaluation']

In [ ]:
x[x['id']=='FOODS_3_776_WI_3_evaluation']

In [ ]:
X_df[X_df['id']=='FOODS_3_776_WI_3_evaluation']

In [ ]:
!python --version

In [38]:
cal_dtypes = {
    'd': 'category',
    'wm_yr_wk': np.uint16,
    'event_name_1': 'category',
    'event_type_1': 'category',
    'event_name_2': 'category',
    'event_type_2': 'category',
    'snap_CA': np.uint8,
    'snap_TX': np.uint8,
    'snap_WI': np.uint8,
}
cal = pd.read_csv( 'calendar.csv', 
                  dtype=cal_dtypes, 
                  usecols=list(cal_dtypes.keys()) + ['date'], 
                  parse_dates=['date'])
event_cols = [k for k in cal_dtypes if k.startswith('event')]
for col in event_cols:
    cal[col] = cal[col].cat.add_categories('nan').fillna('nan')

In [39]:
prices_dtypes = {
    'store_id': 'category',
    'item_id': 'category',
    'wm_yr_wk': np.uint16,
    'sell_price': np.float32
}
prices = pd.read_csv('sell_prices.csv', dtype=prices_dtypes)

In [40]:
sales_dtypes = {
    'id': 'category',
    'item_id': prices.item_id.dtype,
    'dept_id': 'category',
    'cat_id': 'category',
    'store_id': 'category',
    'state_id': 'category',
    **{f'd_{i}': np.float32 for i in range(1942)}
}
sales = pd.read_csv(
    'sales_train_evaluation.csv',
    dtype=sales_dtypes,
)

In [42]:
long = sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

In [43]:
%%time
print(long.shape[0])
long['date_idx'] = long['d'].str.replace('d_', '').astype('int32')
dates = sorted(long['date_idx'].unique())
long = long.sort_values(['id', 'date_idx'])


59181090
Wall time: 1min


In [44]:
without_leading_zeros = long['y'].gt(0).groupby(long['id']).transform('cummax')

In [45]:
above_min_date = long['date_idx'] >= dates[-400]
keep_mask = without_leading_zeros & above_min_date
long = long[keep_mask]
print(long.shape[0])

12159132


In [46]:
long['d'] = long['d'].astype(cal.d.dtype)


In [47]:
long = long.merge(cal, on=['d'])
long = long.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'])
last_wmyrwk = long['wm_yr_wk'].max()
last_date_train = long['date'].max()
long = long.drop(columns=['d', 'date_idx', 'wm_yr_wk'])
long = long.sample(frac=1.0, random_state=0).reset_index(drop=True)

# build future X
future_cal = cal[cal['date'] > last_date_train]
future_prices = prices[prices['wm_yr_wk'] >= last_wmyrwk].copy()
future_prices['id'] = future_prices['item_id'].astype(str) + '_' + future_prices['store_id'].astype(str) + '_evaluation'
X_df = future_prices.merge(future_cal, on='wm_yr_wk').drop(columns=['store_id', 'item_id', 'wm_yr_wk', 'd'])

In [ ]:
model_params = {
    'verbose': -1,
    'force_col_wise': True,
    'num_leaves': 256,
    'n_estimators': 50,
}

fcst = MLForecast(
    models=[lgb.LGBMRegressor(**model_params)],
    freq='D',
    lags=[7 * (i+1) for i in range(8)],
    lag_transforms = {
        1:  [ExpandingMean()],
        7:  [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        14: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
        28: [RollingMean(7), RollingMean(14), RollingMean(28), SeasonalRollingMean(7, 4)],
    },
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week'],    
    num_threads=4,
)